In [1]:
#PERMANENT CELL 1

import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
import pickle
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import clear_output
import math
import sqlite3
from Calibration_Variables import *
import copy
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA


In [68]:
for event in events:
    print (event[0] + ' - ' + event[1])

2021-11-12 - 2021-11-17
2021-09-16 - 2021-09-20
2021-10-14 - 2021-10-19
2021-11-24 - 2021-11-30
2022-03-12 - 2022-03-17
2022-02-25 - 2022-03-02


In [62]:
#Make sure to add them in prioritized order, most important at the top.
start_day_chart_delay = 1

events = []
events.append(['2021-11-12','2021-11-17',['BN21','CV14','ML1','ML2','ML1-2','NW2','NW2B']])
events.append(['2021-09-16','2021-09-20',['BI1','NW2','NW2A','NW2B']])
events.append(['2021-10-14','2021-10-19',['AL3','NW2','NW2B']])
events.append(['2021-11-24','2021-11-30',['NW2','NW2B']])
events.append(['2022-03-12','2022-03-17',['NRT20']])
events.append(['2022-02-25','2022-03-02',[]])

In [10]:
specs_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Calibration_2022\WWF_Calibration_Reports\FSA_Calibration_Specs"


rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Calibration_2022\DATA\RAINFALL\FSA_Rainfall_Data_PDT.dfs0"
map_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Calibration_2022\Calibration_Reports\FSA_Report_Maps"

dfs0_folders = []
dfs0_folders.append(r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\04_DATA\11.Dfs0\Non-PS")
dfs0_folders.append(r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\04_DATA\11.Dfs0\PS")


In [63]:
#Import dfs0 files

#Import rain gauges
rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()
warnings = []

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':


            if 'ps' in f.lower() and f[:2].lower() != 'ps':
                gauge_id = re.split(r'[.]',f)[0]
            else:
                gauge_id = re.split(r'_|[.]',f)[0]
            res = mikeio.read(dfs0_folder + '\\' + f)
            ts = res.to_dataframe()

            gauge_ids.append(gauge_id)

            first_level = True
            second_level = True
            first_velocity = True
            for i, column in enumerate(ts.columns):
                if i == 0:
                    ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
                    if 'meter pow 3 per sec' in str(res.items[0]):
                        ts.Flow = ts.Flow * 1000
                    elif 'liter per sec' in str(res.items[0]):
                        ts.Flow = ts.Flow 
                    else:
                        warnings.append('First item in ' + f + ' does not appear to be type Discharge. This is not imported.')
                        ts.Flow = np.nan

                elif str(res.items[i])[-21:] == '<Water Level> (meter)': 
                    if first_level == True:
                        ts.rename(columns={ts.columns[i]:'Level'},inplace=True)
                        first_level = False
                    elif second_level == True:
                        ts.rename(columns={ts.columns[i]:'Level2'},inplace=True)
                        second_level = False


                elif str(res.items[i])[-31:] == '<Flow velocity> (meter per sec)' and first_velocity == True:
                    ts.rename(columns={ts.columns[i]:'Velocity'},inplace=True)
                    first_velocity = False

            if not 'Flow' in ts.columns:
                ts['Flow'] = np.nan                     
            if not 'Velocity' in ts.columns:
                ts['Velocity'] = np.nan
            if not 'Level' in ts.columns:
                ts['Level'] = np.nan
            if not 'Level2' in ts.columns:
                ts['Level2'] = np.nan

            ts['Gauge'] = gauge_id
            ts = ts[['Gauge','Flow','Level','Level2','Velocity']]
            ts['Seconds'] = ts.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
            ts['Volume'] = ts.Flow * ts.Seconds / 1000
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False




In [73]:
columns = ['Gauge']
event_number = len(events)
for i in range(event_number):
    columns.append('Start' + str(i+1))
    columns.append('End' + str(i+1))
columns

['Gauge',
 'Start1',
 'End1',
 'Start2',
 'End2',
 'Start3',
 'End3',
 'Start4',
 'End4',
 'Start5',
 'End5',
 'Start6',
 'End6']

In [75]:
all_gauge_df = pd.read_excel(specs_folder + '\\Calibration_Specifications.xlsx', sheet_name="Periods")
gauges = list(all_gauge_df.Meter.unique())

columns = ['Gauge']
event_number = len(events)
for i in range(event_number):
    columns.append('Start' + str(i+1))
    columns.append('End' + str(i+1))

measured_simple = measured[['Gauge']].copy()
measured_simple['DateTime'] = measured_simple.index
measured_agg = measured_simple.groupby('Gauge').agg(From=('DateTime', 'min'), To=('DateTime', 'max'))
gauges_events = []

for gauge in gauges:    
    gauge_events = [gauge]
    event_no = 0
    standby_events = events.copy()
    if gauge in measured_agg.index:
        for i, event in enumerate(events):
            start = pd.to_datetime(event[0])
            end = pd.to_datetime(event[1])
            exclusions = event[2]
            if start >= measured_agg.loc[gauge,'From'] and end <= measured_agg.loc[gauge,'To'] and not gauge in exclusions:
                gauge_events.append(start + dt.timedelta(days=start_day_chart_delay))
                gauge_events.append(end)
                del standby_events[i + len(standby_events) - len(events)]
                event_no += 1
                if event_no >=event_number or i == len(events)-1:
                    break
    if event_no < event_number:
        for i in range(event_number-event_no):
            start = pd.to_datetime(standby_events[i][0])
            end = pd.to_datetime(standby_events[i][1])
            gauge_events.append(start + dt.timedelta(days=start_day_chart_delay))
            gauge_events.append(end)
    gauges_events.append(gauge_events)
                
gauges_events_df = pd.DataFrame(gauges_events,columns=columns)
gauges_events_df.to_csv(specs_folder + '\\Chart_Dates.csv',index=False)
            
gauges_events_df            
                   

,Gauge,Start1,End1,Start2,End2,Start3,End3,Start4,End4,Start5,End5,Start6,End6
0,20th_Street_PS,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02
1,AL1,2021-09-17,2021-09-20,2021-11-13,2021-11-17,2021-10-15,2021-10-19,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02
2,AL3,2021-11-13,2021-11-17,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02,2021-09-17,2021-09-20,2021-10-15,2021-10-19
3,AN1,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02
4,BI1,2021-11-13,2021-11-17,2021-10-15,2021-10-19,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02,2021-09-17,2021-09-20
5,BIN18,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19
6,BN21,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19
7,BNC9,2022-03-13,2022-03-17,2022-02-26,2022-03-02,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19,2021-11-25,2021-11-30
8,BNP17,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02
9,CQ1,2021-11-13,2021-11-17,2021-09-17,2021-09-20,2021-10-15,2021-10-19,2021-11-25,2021-11-30,2022-03-13,2022-03-17,2022-02-26,2022-03-02
